In [1]:
#first we import libraries to use
import pandas as pd
from google.colab import auth
import pandas_gbq
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import random as random
from scipy import stats

# Will collect credentials
auth.authenticate_user()

In [10]:
query = "SELECT * FROM `anime-forecasting.main_table_joined_not_exploaded.left_join_mega_table_exclude_eps_not_exploaded`"

In [35]:
table = pandas_gbq.read_gbq(query, project_id="anime-forecasting")

Downloading: 100%|██████████|


#Machine Learning - Clustering

In [4]:
#scalers
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
#encoders
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [5]:
#neighbors
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans

In [6]:
#validating
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict

In [40]:
table.drop_duplicates(subset='idMal',inplace=True)

In [41]:
table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2521 entries, 0 to 2537
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   idMal                      2521 non-null   Int64  
 1   title                      2521 non-null   object 
 2   format                     2521 non-null   object 
 3   episodes                   2521 non-null   Int64  
 4   duration                   2521 non-null   Int64  
 5   status                     2521 non-null   object 
 6   startDay                   2521 non-null   Int64  
 7   startMonth                 2521 non-null   Int64  
 8   startYear                  2521 non-null   Int64  
 9   meanScore                  2521 non-null   float64
 10  favourites                 2521 non-null   Int64  
 11  genres                     2521 non-null   object 
 12  scoreDistribution_score    2521 non-null   Int64  
 13  statusDistribution_score   2521 non-null   Int64  
 1

In [27]:
table['idMal'].nunique()

2521

In [13]:
table.columns

Index(['idMal', 'title', 'format', 'episodes', 'duration', 'status',
       'startDay', 'startMonth', 'startYear', 'meanScore', 'favourites',
       'genres', 'scoreDistribution_score', 'statusDistribution_score',
       'statusDistribution_status', 'rankings_context', 'tags', 'start_date',
       'external_sites', 'Score', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Popularity', 'Favorites', 'Scored_by'],
      dtype='object')

In [42]:
table = table.drop(['format','startDay', 'startMonth', 'startYear','scoreDistribution_score', 'statusDistribution_score',
       'statusDistribution_status', 'rankings_context','start_date',
       'external_sites', 'Score', 'Premiered', 'Producers', 'Licensors',
       'Studios','Scored_by'], axis=1)

In [43]:
table = table.drop(['Popularity', 'Favorites'], axis=1)

In [17]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2538 entries, 0 to 2537
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   idMal       2538 non-null   Int64  
 1   title       2538 non-null   object 
 2   episodes    2538 non-null   Int64  
 3   duration    2538 non-null   Int64  
 4   status      2538 non-null   object 
 5   meanScore   2538 non-null   float64
 6   favourites  2538 non-null   Int64  
 7   genres      2538 non-null   object 
 8   tags        2538 non-null   object 
dtypes: Int64(4), float64(1), object(4)
memory usage: 188.5+ KB


In [44]:
table['genres'] = table['genres'].apply(lambda x: x.strip('][').split(', '))

In [45]:
table.explode('genres')

,idMal,title,episodes,duration,status,meanScore,favourites,genres,tags
0,55848,Suicide Squad ISEKAI,10,24,FINISHED,63.0,841,'Action',"['Isekai', 'Anti-Hero', 'Crime', 'Crossover', ..."
0,55848,Suicide Squad ISEKAI,10,24,FINISHED,63.0,841,'Adventure',"['Isekai', 'Anti-Hero', 'Crime', 'Crossover', ..."
0,55848,Suicide Squad ISEKAI,10,24,FINISHED,63.0,841,'Fantasy',"['Isekai', 'Anti-Hero', 'Crime', 'Crossover', ..."
0,55848,Suicide Squad ISEKAI,10,24,FINISHED,63.0,841,'Sci-Fi',"['Isekai', 'Anti-Hero', 'Crime', 'Crossover', ..."
1,35333,Hitori no Shita - The Outcast 2,24,24,FINISHED,71.0,218,'Action',"['Super Power', 'Urban Fantasy', 'Martial Arts..."
...,...,...,...,...,...,...,...,...,...
2536,960,Legend of the Condor Hero II,26,20,FINISHED,68.0,6,'Adventure',['Historical']
2536,960,Legend of the Condor Hero II,26,20,FINISHED,68.0,6,'Drama',['Historical']
2536,960,Legend of the Condor Hero II,26,20,FINISHED,68.0,6,'Romance',['Historical']
2537,50185,Salaryman's Club,12,23,FINISHED,72.0,338,'Drama',"['Badminton', 'Work', 'Primarily Male Cast', '..."


In [50]:
table.set_index('idMal',inplace=True)

In [56]:
from collections import Counter
genres_table = (table
 .set_index('idMal')
 .genres
 .map(Counter)
 .apply(pd.Series)
 .fillna(0, downcast='infer')
 .reset_index()
)

<ipython-input-56-3c1f32bd4bb0>:2: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  genres_table = (table


In [57]:
table['tags'] = table['tags'].apply(lambda x: x.strip('][').split(', '))

In [58]:
tags_table = (table
 .set_index('idMal')
 .tags
 .map(Counter)
 .apply(pd.Series)
 .fillna(0, downcast='infer')
 .reset_index()
)

<ipython-input-58-612219a1201d>:1: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  tags_table = (table


In [59]:
tags_table

,idMal,'Isekai','Anti-Hero','Crime','Crossover','Female Protagonist','Super Power','War','Magic','Primarily Adult Cast',...,'Irrumatio','Virginity','Jazz Music','Nakadashi','Anal Sex','Defloration','Flat Chest','Blackmail','Asphyxiation','Sex Toys'
0,55848,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,35333,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,54492,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,43989,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,44863,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,257,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2517,1142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2518,16353,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2519,960,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
tags_table.value_counts()

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,count
idMal,'Isekai','Anti-Hero','Crime','Crossover','Female Protagonist','Super Power','War','Magic','Primarily Adult Cast','Conspiracy','Guns','Prison','Language Barrier','Shapeshifting','Meta','Swordplay','Primarily Male Cast','Henshin','Dragons','CGI','Necromancy','Skeleton','Elf','Heterosexual','Nudity','Urban Fantasy','Martial Arts','Male Protagonist','Wuxia','Medicine','Historical','Detective','Foreign','Politics','Ancient China','Drugs','Work','Royal Affairs','Adoption','Unrequited Love','Alchemy','Maids','Tragedy','Food','Slavery','Chibi','Asexual','Episodic','Go',,'Cars','Classic Literature','Motorcycles','Kids','Time Manipulation','Travel','Ships','Ghost','Primarily Child Cast','School','Youkai','Afterlife','Parody','Demons','Animals','Ensemble Cast','Tanned Skin','Found Family','Military','Robots','Aliens','Teacher','Seinen','Time Skip','Space','Environmental','Slapstick','Rural','Outdoor','Super Robot','Proxy Battle','Denpa','Dissociative Identities','Cosmic Horror','Rehabilitation','Cult','Mythology','Bullying','Philosophy','Love Triangle','Kuudere','Card Battle','Idol','Cute Girls Doing Cute Things','Amnesia','Gore','Survival','Body Horror','Dancing','Vampire','Coming of Age','Incest','Feet','Snowscape','Primarily Female Cast','Achronological Order','Gods','Primarily Teen Cast','Post-Apocalyptic','School Club','Memory Manipulation','Religion','Drawing','Photography','Filmmaking','Archery','Cute Boys Doing Cute Things','Tsundere','Twins','Anachronism','Bisexual','Video Games','Full CGI','Urban','Mopeds','Disability','Dungeon','Kemonomimi','Zombie','Veterinarian','Reincarnation','Anthology','Anthropomorphism','Shounen','Cultivation','Superhero','Gender Bending','Revenge','Cyborg','Dinosaurs','Body Swapping','Battle Royale','Police','Wrestling','Surreal Comedy','Dystopian','Real Robot','Shoujo','Tomboy','Tokusatsu','LGBTQ+ Themes','Age Gap','Yuri','Alternate Universe','Shrine Maiden','Makeup','Gambling','Volleyball','Female Harem','College','Death Game','Fairy Tale','Terrorism','MILF','Boarding School','Arranged Marriage','Band','Hip-hop Music','Bar','Rock Music','Fugitive','Monster Boy','Estranged Family','Kingdom Management','Primarily Animal Cast','Kaiju','Artificial Intelligence','Delinquents','Cyberpunk','Monster Girl','Yandere','Rape','Family Life','Coastal','Iyashikei','Mafia','Trains','Noir','Vore','Samurai','Cannibalism','Mermaid','Watersports','Bondage','Masochism','Tentacles','Gyaru','Chuunibyou','Musical Theater','Goblin','Rugby','Educational','Orphan','Ninja','Transgender','Clone','Lost Civilization','Pandemic','Angels','Aviation','Classical Music','Ice Skating','Athletics','Butler','Psychosexual','Board Game','Villainess','Josei','Ojou-sama','Shogi','Parkour','Achromatic','Homeless','Espionage','Suicide','Nun','Stop Motion','Otaku Culture','Astronomy',"""Boys' Love""",'Virtual World','Crossdressing','Restaurant','Torture','Time Loop','Witch','Assassins','Large Breasts','Fitness','Agriculture','Fairy','Nekomimi','Fake Relationship','Parenthood','Tennis','Football','Boxing','Femboy','Natural Disaster','Space Opera','Desert','Steampunk','Fashion','Pirates','Male Harem','Class Struggle','Matriarchy','Cheerleading','Age Regression','Werewolf','Satire','Baseball','Spearplay','Sadism','Poker','Lacrosse','Acting','Polyamorous','Chimera','Economics','Gangs','Succubus','Fishing','Aromantic','Writing','Calligraphy','Agender','Dullahan','Basketball','Mixed Gender Harem','Hikikomori','Cosplay','Yakuza','Circus','Inn','Advertisement','POV','Badminton','Marriage','Criminal Organization','Swimming','Human Pet','Tanks','Femdom','Rotoscoping','Rakugo','Masturb

In [69]:
tags_remove = pd.DataFrame(tags_table.sum(axis=0))

In [70]:
tags_remove.columns

RangeIndex(start=0, stop=1, step=1)

In [77]:
col_remove =list(tags_remove[tags_remove[0] < 6].index)

In [79]:
tags_table=tags_table.drop(columns=col_remove)

In [82]:
joined_table = pd.concat([table,genres_table,tags_table],axis=1)

In [83]:
joined_table = joined_table.drop(['genres','tags'], axis=1)

In [86]:
joined_table

,idMal,title,episodes,duration,status,meanScore,favourites,idMal,'Action','Adventure',...,'Airsoft','Inseki','Augmented Reality','Karuta','Netorare','VTuber','Cycling','Centaur','American Football','Elderly Protagonist'
0,55848,Suicide Squad ISEKAI,10,24,FINISHED,63.0,841,55848,1,1,...,0,0,0,0,0,0,0,0,0,0
1,35333,Hitori no Shita - The Outcast 2,24,24,FINISHED,71.0,218,35333,1,0,...,0,0,0,0,0,0,0,0,0,0
2,54492,The Apothecary Diaries,24,23,FINISHED,88.0,10978,54492,0,0,...,0,0,0,0,0,0,0,0,0,0
3,43989,Pleasant Goat and Big Big Wolf: Paddi the Amaz...,52,25,FINISHED,35.0,0,43989,0,1,...,0,0,0,0,0,0,0,0,0,0
4,44863,Magic Wonderland,26,22,FINISHED,51.0,0,44863,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,257,Battle Vixens: Ikki Tousen,13,24,FINISHED,57.0,190,257,1,0,...,0,0,0,0,0,0,0,0,0,0
2517,1142,Honey and Clover II,12,23,FINISHED,79.0,273,1142,0,0,...,0,0,0,0,0,0,0,0,0,0
2518,16353,Love Lab,13,24,FINISHED,71.0,331,16353,0,0,...,0,0,0,0,0,0,0,0,0,0
2519,960,Legend of the Condor Hero II,26,20,FINISHED,68.0,6,960,0,1,...,0,0,0,0,0,0,0,0,0,0


In [90]:
model_table = joined_table.drop(['idMal','duration','title','status','favourites'], axis=1)

In [52]:
# pd.crosstab(index = table.index,columns=table.explode('genres')['genres'])

ValueError: operands could not be broadcast together with shapes (2521,) (7971,) (2521,) 

In [47]:
new_table = table.groupby('idMal').count()

In [48]:
new_table[new_table['title']>1]

,title,episodes,duration,status,meanScore,favourites,genres,tags
idMal,,,,,,,,


In [54]:
table.reset_index(inplace=True)

In [33]:
table[table['idMal']== 2521]

,idMal,title,episodes,duration,status,meanScore,favourites,genres,tags
2153,33089,Kemono Friends,12,24,FINISHED,71.0,533,"['Adventure', 'Comedy', 'Fantasy']","['Kemonomimi', 'Full CGI', 'Post-Apocalyptic',..."
2154,33089,Kemono Friends,12,24,FINISHED,71.0,533,"['Adventure', 'Comedy', 'Fantasy']","['Kemonomimi', 'Full CGI', 'Post-Apocalyptic',..."
2155,33089,Kemono Friends,12,24,FINISHED,71.0,533,"['Adventure', 'Comedy', 'Fantasy']","['Kemonomimi', 'Full CGI', 'Post-Apocalyptic',..."


TypeError: unhashable type: 'list'

In [28]:
pd.crosstab(index=table.index, columns=table.explode('genres')['genres'])

ValueError: operands could not be broadcast together with shapes (2538,) (8026,) (2538,) 

In [7]:
table['genres'].value_counts()

,count
genres,
Action,121298
Fantasy,98767
Comedy,95727
Drama,87446
Adventure,83075
Supernatural,58166
Romance,57770
Sci-Fi,45638
Slice of Life,43644


In [8]:
table['tags'].value_counts()

,count
tags,
Male Protagonist,31174
Female Protagonist,20006
Tragedy,16765
Heterosexual,16531
Magic,16182
...,...
Autobiographical,10
Handball,10
Omegaverse,9


In [ ]:
temp = table.dropna()

In [ ]:
temp.groupby(by='idMal').count().sort_values(by='title', ascending=False)

In [ ]:
for col in ["meanScore", "Popularity", "favourites"]:
    print(f"{col}\t>> {temp[col].nunique()} number of unique categories.")

In [ ]:
# group = temp.groupby(by='article_id').first().drop(columns=['age', 'price', 'customer_id']).reset_index()

In [ ]:
# group[group['product_type_name'] == "Dress"]

In [ ]:
genre_tags_pivot = table.pivot(index=['title', 'idMal'], columns=['genres','tags'], value=0)

In [ ]:
# non_numerical_cols = all_art.select_dtypes(include=['object']).columns
# encoder = OneHotEncoder(sparse=False, drop=None)
# encoded_data = encoder.fit_transform(all_art[non_numerical_cols])

# encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(non_numerical_cols))

# df = pd.concat([all_art.drop(columns=non_numerical_cols), encoded_df], axis=1)

#Predicting

In [91]:
model_table

,episodes,meanScore,'Action','Adventure','Fantasy','Sci-Fi','Supernatural','Drama','Mystery','Comedy',...,'Airsoft','Inseki','Augmented Reality','Karuta','Netorare','VTuber','Cycling','Centaur','American Football','Elderly Protagonist'
0,10,63.0,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,24,71.0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,24,88.0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,52,35.0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,26,51.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,13,57.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2517,12,79.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2518,13,71.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2519,26,68.0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
joined_table

,idMal,title,episodes,duration,status,meanScore,favourites,idMal,'Action','Adventure',...,'Airsoft','Inseki','Augmented Reality','Karuta','Netorare','VTuber','Cycling','Centaur','American Football','Elderly Protagonist'
0,55848,Suicide Squad ISEKAI,10,24,FINISHED,63.0,841,55848,1,1,...,0,0,0,0,0,0,0,0,0,0
1,35333,Hitori no Shita - The Outcast 2,24,24,FINISHED,71.0,218,35333,1,0,...,0,0,0,0,0,0,0,0,0,0
2,54492,The Apothecary Diaries,24,23,FINISHED,88.0,10978,54492,0,0,...,0,0,0,0,0,0,0,0,0,0
3,43989,Pleasant Goat and Big Big Wolf: Paddi the Amaz...,52,25,FINISHED,35.0,0,43989,0,1,...,0,0,0,0,0,0,0,0,0,0
4,44863,Magic Wonderland,26,22,FINISHED,51.0,0,44863,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,257,Battle Vixens: Ikki Tousen,13,24,FINISHED,57.0,190,257,1,0,...,0,0,0,0,0,0,0,0,0,0
2517,1142,Honey and Clover II,12,23,FINISHED,79.0,273,1142,0,0,...,0,0,0,0,0,0,0,0,0,0
2518,16353,Love Lab,13,24,FINISHED,71.0,331,16353,0,0,...,0,0,0,0,0,0,0,0,0,0
2519,960,Legend of the Condor Hero II,26,20,FINISHED,68.0,6,960,0,1,...,0,0,0,0,0,0,0,0,0,0


In [92]:
# Define X and y
X = model_table #Remove non numerical features
y = model_table['meanScore'] #"meanScore", "Popularity", "favourites" -- possible y columns?

# Scale the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

knn_model = KNeighborsRegressor().fit(X_scaled,y) # Instanciate and train model

In [93]:
knn_stored = {}
knn_list = []

In [119]:
def find_nearest(idMal):
    index = np.where(joined_table['idMal'] == idMal)[0][0]
    test_item = joined_table.iloc[index:(index+1)]
    X_new = test_item.drop(['idMal','duration','title','status','favourites'], axis=1)
    X_new_scaled = scaler.transform(X_new)
    ind_list = list(knn_model.kneighbors(X_new_scaled,n_neighbors=10)[1][0])[1:]
    ids = joined_table.iloc[ind_list]
    knn_list.append(ids)
    return ids


In [140]:
find_nearest(6702) #20583 - haikyuu, #20 - naruto, 1535 - death note

,idMal,title,episodes,duration,status,meanScore,favourites,idMal,'Action','Adventure',...,'Airsoft','Inseki','Augmented Reality','Karuta','Netorare','VTuber','Cycling','Centaur','American Football','Elderly Protagonist'
940,22043,Fairy Tail Series 2,102,24,FINISHED,74.0,3921,22043,1,1,...,0,0,0,0,0,0,0,1,0,0
385,1017,Orphen,24,25,FINISHED,67.0,90,1017,1,1,...,0,0,0,0,0,0,0,0,0,0
1367,35972,Fairy Tail Final Season,51,24,FINISHED,75.0,3418,35972,1,1,...,0,0,0,0,0,0,0,0,0,0
679,1172,The Slayers Try,26,23,FINISHED,74.0,126,1172,0,1,...,0,0,0,0,0,0,0,0,0,0
320,50002,EDENS ZERO Season 2,25,23,FINISHED,73.0,466,50002,1,1,...,0,0,0,0,0,0,0,0,0,0
1597,51794,The Seven Deadly Sins: Four Knights of the Apo...,24,24,FINISHED,71.0,453,51794,1,1,...,0,0,0,0,0,0,0,0,0,0
2046,53446,Campfire Cooking in Another World with my Absu...,12,24,FINISHED,76.0,2161,53446,0,1,...,0,0,0,0,0,0,0,0,0,0
1709,9539,Cardfight Vanguard,65,24,FINISHED,66.0,180,9539,1,1,...,0,0,0,0,0,0,0,0,0,0
2253,246,Rave Master,51,24,FINISHED,67.0,174,246,0,1,...,0,0,0,0,0,0,0,0,0,0
